In [9]:
manifest = {
    'memory': 256,
    'disk_quota': 256,
    'buildpack': 'python_buildpack',
    "requirements":[
        "pandas",
        "afs"
    ],
    'type': 'APP'
}


In [10]:
from datetime import datetime, timedelta
import json
import csv
import requests
import pandas as pd
from influxdb import InfluxDBClient
import numpy as np
import time
from afs import services

In [12]:
train_data="""
,STATUS_EQUIPMENT,STATUS_FAN,VOLTAGE_INPUT,PRESSURE_OUTPUT,KW_FAN,KW_EQUIPMENT,KW_SUMMARY,TEMPERATURE_ENVIRONMENT,TEMPERATURE_OUTPUT
0,0,1,217.5,1.65,2.5,36.54,31.54,0.0,83
1,0,0,217.5,6.05,3.8,30.22,29.99,0.0,84
2,0,1,213.6,5.3,0.85,38.12,24.96,0.0,85
3,0,0,213.6,5.75,0.4,38.51,23.03,0.0,86
4,0,0,214.4,3.85,1.15,37.72,27.28,0.0,86
5,0,0,217.5,6.05,3.8,30.22,29.99,0.0,90
6,0,0,215.2,5.85,1.15,36.54,36.18,0.0,90
7,0,1,213.6,6.25,0.4,38.51,22.64,0.0,90
8,0,1,214.4,6.25,0.4,38.51,22.64,0.0,92
9,0,0,217.5,2.75,0.65,32.59,37.73,0.0,94
10,1,1,215.2,3.75,0.85,38.12,25.35,0.0,94
11,1,1,215.2,1.65,3.8,37.33,29.61,0.0,97
12,1,1,215.95,6.95,0.4,37.33,31.93,0.0,98
13,1,1,214.4,2.05,0.85,38.12,25.35,0.0,98
14,0,0,215.2,6.95,0.85,36.93,33.86,0.0,98
15,0,1,215.95,7.05,0.65,30.61,34.64,0.0,98
16,0,1,213.6,3.85,0.4,38.51,22.64,0.0,99
17,1,0,215.95,5.65,0.4,31.4,29.61,0.0,99
18,1,1,213.6,2.35,1.0,22.71,23.03,0.0,100
19,0,1,218.3,6.85,0.15,36.54,33.48,0.0,100
20,1,0,214.4,2.25,1.15,37.72,27.67,0.0,83
21,1,1,217.5,6.25,2.5,38.91,37.73,0.0,84
22,1,1,212.8,1.75,0.85,23.5,24.19,0.0,84
23,0,1,213.6,5.3,0.85,38.12,24.96,0.0,86
24,1,1,218.7,5.75,0.4,25.48,4.06,0.0,86
25,0,0,215.2,6.25,1.15,36.54,36.18,0.0,86
26,1,1,212.8,1.75,0.85,31.4,32.7,0.0,88
27,1,1,215.95,1.65,3.8,37.33,29.61,0.0,88
28,1,1,213.6,2.35,1.0,22.71,23.03,0.0,90
29,1,0,213.6,3.5,3.8,38.51,20.32,0.0,90
30,0,0,214.4,3.85,0.4,38.51,23.03,0.0,92
31,0,1,213.6,5.3,0.85,38.12,24.96,0.0,95
32,1,0,214.4,3.65,0.4,38.51,22.25,0.0,96
33,0,1,215.95,2.85,3.8,37.33,26.9,0.0,96
34,1,0,218.3,5.55,0.15,36.54,33.09,0.0,97
35,1,0,212.8,5.75,1.0,32.59,33.48,0.0,97
36,1,1,215.95,7.15,1.0,38.51,31.93,0.0,98
37,1,1,215.95,6.95,1.15,36.54,36.18,0.0,98
38,1,1,215.95,6.35,3.8,36.14,38.12,0.0,100
39,1,1,215.2,5.65,0.4,37.33,31.93,0.0,100
40,1,0,215.95,7.05,0.65,35.75,36.96,0.0,83
41,1,1,215.2,6.15,0.85,38.12,25.35,0.0,84
42,0,1,217.5,5.75,1.0,31.4,27.28,0.0,84
43,0,0,213.6,2.65,0.85,38.12,25.35,0.0,84
44,1,1,214.4,6.95,0.4,20.34,22.25,0.0,84
45,1,1,215.2,3.65,0.85,38.12,25.35,0.0,85
46,1,1,215.95,2.05,0.85,36.93,33.86,0.0,86
47,0,0,215.95,2.05,1.15,36.54,36.18,0.0,86
48,1,1,215.95,7.15,1.0,38.51,31.93,0.0,88
49,1,0,212.8,4.9,0.15,35.75,32.31,0.0,88
50,0,0,215.95,2.75,0.4,37.33,31.93,0.0,90
51,0,0,215.95,5.1,3.8,36.14,38.12,0.0,90
52,0,1,215.2,6.15,0.4,37.33,31.54,0.0,90
53,0,0,215.95,3.75,0.4,37.33,31.93,0.0,91
54,0,0,214.4,6.15,0.4,38.51,23.03,0.0,94
55,1,1,216.7,6.45,0.15,34.17,31.54,0.0,96
56,1,0,214.4,2.25,1.15,37.72,27.67,0.0,97
57,0,0,213.6,6.75,3.8,38.51,21.09,0.0,98
58,1,0,215.95,3.65,0.85,26.66,21.48,0.0,100
59,0,1,213.6,2.65,1.0,21.13,30.38,0.0,100
60,0,0,213.6,3.75,0.4,37.33,31.93,0.0,83
61,1,1,215.95,5.65,0.4,37.33,31.93,0.0,84
62,0,0,215.95,7.4,3.8,38.91,29.22,0.0,84
63,0,0,212.8,6.05,0.65,32.59,34.64,0.0,84
64,1,0,218.7,4.5,1.15,26.66,34.25,0.0,85
65,1,1,214.4,3.35,0.85,38.12,25.35,0.0,86
66,1,1,215.2,1.65,3.8,37.33,29.61,0.0,86
67,0,0,213.6,2.75,1.15,37.72,27.28,0.0,86
68,0,1,213.6,5.3,0.85,38.12,24.96,0.0,88
69,0,0,212.8,3.15,0.85,36.54,0.19,0.0,90
70,1,1,214.4,6.15,1.15,37.72,27.28,0.0,90
71,1,1,214.4,6.05,0.85,38.12,25.35,0.0,90
72,0,0,215.2,3.85,1.15,37.72,27.28,0.0,90
73,1,1,215.95,3.35,1.15,36.54,36.18,0.0,92
74,0,0,215.2,3.85,1.15,36.54,36.18,0.0,96
75,1,1,214.4,6.15,0.4,38.51,23.03,0.0,98
76,1,0,215.95,3.5,3.8,29.43,29.22,0.0,98
77,0,0,215.95,5.1,3.8,36.14,38.12,0.0,100
78,0,0,213.6,6.45,0.85,38.12,25.35,0.0,100
79,0,1,213.6,5.45,3.8,38.51,20.7,0.0,100
80,0,1,213.6,6.25,0.4,38.51,22.64,0.0,84
81,0,0,215.2,3.35,1.15,37.72,27.28,0.0,84
82,1,1,214.4,1.65,3.8,20.34,19.93,0.0,84
83,1,0,214.4,3.15,0.85,30.22,23.41,0.0,86
84,1,0,216.7,2.05,1.15,19.16,26.9,0.0,86
85,1,1,215.95,3.75,0.85,36.93,33.86,0.0,87
86,0,1,214.4,1.95,1.0,32.19,26.12,0.0,90
87,1,1,214.4,1.65,3.8,20.34,19.93,0.0,90
88,0,0,215.2,1.85,0.85,36.93,33.86,0.0,93
89,1,1,215.95,2.85,0.4,37.33,31.93,0.0,93
90,0,1,213.6,5.3,0.85,38.12,24.96,0.0,94
91,0,1,213.6,5.3,0.85,38.12,24.96,0.0,94
92,1,1,215.95,7.4,3.8,36.14,38.12,0.0,94
93,0,0,215.2,1.45,1.15,36.54,36.18,0.0,94
94,1,1,215.95,6.95,0.4,37.33,31.93,0.0,94
95,1,1,215.2,6.95,0.4,37.33,31.93,0.0,96
96,1,1,215.95,1.75,0.85,36.93,33.86,0.0,96
97,0,1,213.6,5.75,0.4,38.51,22.64,0.0,98
98,0,1,216.7,4.3,1.15,31.01,26.12,0.0,98
99,1,0,215.95,3.5,1.15,36.54,36.57,0.0,100
100,1,1,214.4,2.75,0.4,38.51,23.03,0.0,83
101,1,0,214.4,7.35,3.8,27.85,24.57,0.0,84
102,1,1,215.95,7.15,1.0,38.51,31.93,0.0,86
103,0,1,213.6,5.45,3.8,38.51,20.7,0.0,88
104,1,1,215.2,2.65,0.65,29.82,35.41,0.0,90
105,1,1,215.2,1.65,3.8,37.33,29.61,0.0,90
106,1,0,215.2,2.85,3.8,37.33,29.99,0.0,90
107,1,0,215.95,6.95,0.85,36.93,34.25,0.0,92
108,0,0,212.8,6.45,0.4,38.51,23.03,0.0,92
109,0,1,218.7,3.35,1.0,21.92,21.09,0.0,92
110,0,0,218.7,6.65,2.5,21.92,31.93,0.0,92
111,0,0,215.2,6.25,0.85,38.12,25.35,0.0,94
112,0,1,214.4,3.5,1.15,37.72,26.9,0.0,97
113,1,1,214.4,3.75,0.85,38.12,25.35,0.0,97
114,1,1,214.4,2.05,0.85,38.12,25.35,0.0,98
115,0,1,213.6,6.45,0.4,38.51,22.64,0.0,98
116,1,1,212.8,3.85,0.15,19.16,37.35,0.0,98
117,0,0,215.2,5.75,0.4,37.33,31.93,0.0,98
118,1,1,214.4,1.85,0.4,38.51,23.03,0.0,98
119,0,0,214.4,6.45,0.4,38.51,23.03,0.0,100
120,1,1,215.95,2.85,0.85,36.93,33.86,0.0,84
121,1,1,216.7,6.65,2.5,0.2,25.35,0.0,86
122,0,0,213.6,6.45,0.4,38.51,23.03,0.0,87
123,0,0,213.6,2.15,0.85,38.12,25.35,0.0,89
124,1,1,214.4,3.75,0.85,38.12,25.35,0.0,89
125,1,1,214.4,1.65,3.8,20.34,19.93,0.0,90
126,0,1,215.95,4.3,3.8,32.19,36.96,0.0,90
127,1,1,214.4,2.65,0.4,38.51,23.03,0.0,90
128,1,1,218.7,6.25,0.65,20.34,23.41,0.0,90
129,0,1,215.95,2.85,3.8,37.33,26.9,0.0,92
130,0,1,218.7,1.85,1.0,21.92,21.09,0.0,92
131,0,0,215.95,2.15,0.85,36.93,33.86,0.0,93
132,0,0,214.4,3.85,1.15,36.54,36.18,0.0,94
133,1,0,218.3,2.25,1.15,22.71,23.8,0.0,94
134,0,0,215.2,3.35,0.85,36.93,33.86,0.0,94
135,0,1,214.4,5.3,0.85,38.12,24.96,0.0,98
136,1,0,214.4,2.85,3.8,37.33,29.99,0.0,98
137,1,1,214.4,1.75,0.85,38.12,25.35,0.0,98
138,1,1,213.6,3.75,0.4,38.51,23.03,0.0,98
139,1,1,214.4,3.35,0.85,34.17,31.93,0.0,98
140,0,1,214.4,6.35,0.65,29.82,19.54,0.0,84
141,1,0,215.95,3.35,0.85,36.93,34.25,0.0,84
142,1,0,213.6,3.5,3.8,38.51,20.32,0.0,84
143,1,1,215.2,5.75,1.15,37.72,27.28,0.0,86
144,1,0,214.4,2.25,1.15,37.72,27.67,0.0,88
145,1,1,215.2,1.75,0.85,38.12,25.35,0.0,89
146,1,1,214.4,5.65,0.4,38.51,23.03,0.0,90
147,1,0,216.7,1.45,0.4,37.33,32.31,0.0,90
148,1,1,215.95,2.85,0.4,37.33,31.93,0.0,90
149,1,1,218.7,1.45,0.85,36.14,29.99,0.0,90
150,0,0,215.2,6.65,3.8,37.33,29.61,0.0,91
151,1,1,215.95,2.05,0.85,36.93,33.86,0.0,92
152,0,0,215.2,2.05,1.15,36.54,36.18,0.0,92
153,1,0,215.2,5.85,0.65,25.08,23.41,0.0,92
154,1,1,215.95,6.85,0.15,22.71,38.12,0.0,94
155,1,1,218.7,1.65,1.15,27.06,18.38,0.0,94
156,0,0,213.6,6.65,3.8,37.33,29.61,0.0,98
157,0,1,213.6,5.3,0.85,38.12,24.96,0.0,100
158,0,0,217.5,6.05,3.8,28.64,33.48,0.0,100
159,0,1,212.8,1.65,2.5,20.34,29.22,0.0,100
160,0,1,215.95,4.3,1.15,34.17,35.41,0.0,84
161,0,0,215.2,5.1,3.8,36.14,38.12,0.0,86
162,0,1,213.6,5.3,0.85,38.12,24.96,0.0,86
163,0,0,215.2,5.85,1.15,36.54,36.18,0.0,86
164,0,1,213.6,5.45,3.8,38.51,20.7,0.0,88
165,0,0,217.5,6.35,0.65,32.59,37.73,0.0,88
166,1,0,216.7,4.0,0.15,36.54,0.19,0.0,90
167,0,0,215.2,2.85,0.85,36.93,33.86,0.0,90
168,0,1,215.2,4.0,0.15,38.51,37.35,0.0,90
169,0,0,215.2,2.85,0.85,36.93,33.86,0.0,94
170,0,0,214.4,2.95,0.85,36.93,33.86,0.0,94
171,1,0,215.95,5.75,0.15,36.54,0.19,0.0,94
172,1,1,214.4,5.75,0.4,38.51,23.03,0.0,96
173,0,1,218.7,1.65,2.5,28.24,35.02,0.0,96
174,1,1,214.4,1.75,0.85,38.12,25.35,0.0,96
175,1,1,214.4,1.95,1.15,37.72,27.28,0.0,98
176,1,1,214.4,2.55,0.85,38.12,25.35,0.0,98
177,0,1,217.5,5.95,2.5,36.54,31.54,0.0,99
178,1,1,216.7,2.85,0.4,37.33,31.93,0.0,100
179,0,0,216.7,6.25,1.15,27.85,34.64,0.0,100
180,1,1,214.4,6.15,1.15,37.72,27.28,0.0,84
181,0,1,214.4,1.55,0.65,29.82,19.54,0.0,84
182,0,0,215.95,6.65,3.8,22.71,19.93,0.0,84
183,0,0,213.6,6.45,0.4,38.51,23.03,0.0,85
184,0,1,214.4,5.3,0.85,38.12,24.96,0.0,86
185,1,1,215.95,6.95,1.15,36.54,36.18,0.0,86
186,1,1,214.4,3.75,0.4,38.51,23.03,0.0,86
187,0,0,215.2,6.65,3.8,37.33,29.61,0.0,87
188,1,1,216.7,4.5,1.15,36.54,36.18,0.0,88
189,0,0,212.8,7.35,0.65,32.59,34.64,0.0,88
190,1,0,216.7,1.45,0.4,37.33,32.31,0.0,90
191,0,0,215.95,7.4,3.8,38.91,29.22,0.0,91
192,1,1,215.2,7.4,3.8,36.14,38.12,0.0,92
193,1,0,218.7,1.65,2.5,26.66,18.38,0.0,93
194,1,0,213.6,3.5,3.8,38.51,20.32,0.0,93
195,1,0,217.5,3.35,1.15,19.16,26.9,0.0,95
196,1,1,214.4,1.65,3.8,20.34,19.93,0.0,96
197,1,1,212.8,3.75,0.85,29.03,30.77,0.0,98
198,1,1,215.95,4.5,1.15,36.54,36.18,0.0,98
199,1,1,215.95,3.65,0.4,37.33,31.93,0.0,100
200,0,1,213.6,5.3,0.85,38.12,24.96,0.0,84
201,0,1,213.6,5.3,0.85,38.12,24.96,0.0,84
202,1,0,215.2,2.85,3.8,37.33,29.99,0.0,84
203,0,0,213.6,3.85,0.4,37.33,31.93,0.0,85
204,0,0,213.6,2.95,0.85,38.12,25.35,0.0,85
205,0,1,214.4,5.3,0.85,38.12,24.96,0.0,86
206,0,1,215.95,2.45,0.65,30.61,34.64,0.0,88
207,1,1,215.2,1.65,3.8,37.33,29.61,0.0,90
208,1,1,216.7,4.5,1.15,36.54,36.18,0.0,90
209,0,0,217.5,4.15,0.4,21.13,31.15,0.0,90
210,0,0,213.6,2.75,1.15,37.72,27.28,0.0,90
211,1,1,214.4,7.35,3.8,38.51,21.09,0.0,90
212,0,0,214.4,5.75,0.4,38.51,23.03,0.0,92
213,1,0,218.3,3.05,0.65,20.34,38.12,0.0,92
214,1,1,214.4,5.65,0.4,20.34,22.25,0.0,92
215,0,0,214.4,6.45,0.4,30.22,31.15,0.0,95
216,0,0,215.2,5.85,0.4,37.33,31.93,0.0,96
217,1,1,215.95,5.75,0.15,22.71,38.12,0.0,98
218,0,1,213.6,5.3,0.85,38.12,24.96,0.0,99
219,0,1,215.2,1.45,3.8,37.33,29.22,0.0,100
220,0,1,218.7,1.65,2.5,28.24,35.02,0.0,84
221,1,1,215.2,2.65,1.15,37.72,27.28,0.0,84
222,0,0,215.95,6.75,3.8,36.14,38.12,0.0,86
223,1,1,215.2,1.75,0.85,38.12,25.35,0.0,86
224,1,1,214.4,1.65,3.8,20.34,19.93,0.0,88
225,0,1,214.4,5.3,0.85,38.12,24.96,0.0,88
226,0,1,217.5,3.15,2.5,36.54,31.54,0.0,90
227,1,0,213.6,3.5,3.8,38.51,20.32,0.0,90
228,1,1,215.2,5.65,0.4,37.33,31.93,0.0,90
229,0,0,215.2,2.65,0.85,38.12,25.35,0.0,90
230,0,1,213.6,5.3,0.85,38.12,24.96,0.0,90
231,1,1,215.95,2.85,0.85,36.93,33.86,0.0,90
232,1,1,215.2,2.55,0.85,36.93,33.86,0.0,93
233,0,0,213.6,2.85,0.85,38.12,25.35,0.0,94
234,1,1,215.2,3.65,0.4,38.51,23.03,0.0,94
235,1,1,215.95,3.35,1.15,36.54,36.18,0.0,96
236,0,0,213.6,1.55,0.85,38.12,25.35,0.0,98
237,0,1,215.2,6.25,0.4,37.33,31.54,0.0,98
238,0,0,213.6,3.85,0.4,38.51,23.03,0.0,100
"""

with open('training_data.csv', 'w') as f:
    f.write(train_data)

In [13]:
# read config file
afs_ser = services()
credential = afs_ser.get_service_info()['influxdb'][0]

host = credential['host']
port = credential['port']
database = credential['database']
username = credential['username']
password = credential['password']
measurement = 'data_mea'
duration_IDB = 5
client = InfluxDBClient(host, port, username, password, database)  # connect influxdb
while True:
	df = pd.read_csv('training_data.csv')
	data = {}
	for i in range(0, len(df)):
		data['measurement'] = measurement
		tags = {}
		tags['sn'] = 'system_data'
		data['tags'] = tags
		fields = {}
		fields['STATUS_FAN']= int(df.iloc[i]['STATUS_FAN'])
		fields['STATUS_EQUIPMENT']= int(df.iloc[i]['STATUS_EQUIPMENT'])
		fields['TEMPERATURE_OUTPUT']= int(df.iloc[i]['TEMPERATURE_OUTPUT'])
		fields['PRESSURE_OUTPUT']= df.iloc[i]['PRESSURE_OUTPUT']
		fields['VOLTAGE_INPUT']= df.iloc[i]['VOLTAGE_INPUT']
		fields['KW_EQUIPMENT']= df.iloc[i]['KW_EQUIPMENT']
		fields['KW_FAN']= df.iloc[i]['KW_FAN']
		fields['KW_SUMMARY']= df.iloc[i]['KW_SUMMARY']
		fields['TEMPERATURE_ENVIRONMENT']= df.iloc[i]['TEMPERATURE_ENVIRONMENT']
		data['fields'] = fields
		json_body = []
		json_body.append(data)
		print (json_body)
		client.write_points(json_body)
		time.sleep(int(duration_IDB))
	print('end')


[{'measurement': 'data_mea', 'tags': {'sn': 'system_data'}, 'fields': {'STATUS_FAN': 1, 'STATUS_EQUIPMENT': 0, 'TEMPERATURE_OUTPUT': 83, 'PRESSURE_OUTPUT': 1.65, 'VOLTAGE_INPUT': 217.5, 'KW_EQUIPMENT': 36.54, 'KW_FAN': 2.5, 'KW_SUMMARY': 31.54, 'TEMPERATURE_ENVIRONMENT': 0.0}}]
[{'measurement': 'data_mea', 'tags': {'sn': 'system_data'}, 'fields': {'STATUS_FAN': 0, 'STATUS_EQUIPMENT': 0, 'TEMPERATURE_OUTPUT': 84, 'PRESSURE_OUTPUT': 6.05, 'VOLTAGE_INPUT': 217.5, 'KW_EQUIPMENT': 30.22, 'KW_FAN': 3.8, 'KW_SUMMARY': 29.99, 'TEMPERATURE_ENVIRONMENT': 0.0}}]


KeyboardInterrupt: 